In [1]:
import numpy as np
import trimesh
import tqdm 
import random

In [2]:
def uniform_sampling_from_mesh(vertices, faces, sample_num):
    # -------- TODO -----------
    # 1. compute area of each triangles
    # 2. compute probability of each triangles from areas
    # 3. sample N faces according to the probability
    # 4. for each face, sample 1 point
    # Note that FOR-LOOP is not allowed!
    # -------- TODO -----------
    points=vertices[faces]
    p1=points[:,0,:]
    p2=points[:,1,:]
    p3=points[:,2,:]

    # 海伦公式
    a=np.sqrt(np.sum((p1-p2)**2,axis=1))
    b=np.sqrt(np.sum((p1-p3)**2,axis=1))
    c=np.sqrt(np.sum((p2-p3)**2,axis=1))
    p=(a+b+c)/2
    area=np.sqrt(p*(p-a)*(p-b)*(p-c))

    tot_area=np.sum(area)
    prob=area/tot_area

    idx_lst=list(range(faces.shape[0]))
    sample_lst=random.choices(idx_lst,weights=prob,k=sample_num)

    r1=np.random.uniform(low=0,high=1,size=(sample_num,1))
    r2=np.random.uniform(low=0,high=1,size=(sample_num,1))

    p1=p1[sample_lst]
    p2=p2[sample_lst]
    p3=p3[sample_lst]
    uniform_pc=(1-np.sqrt(r1))*p1+np.sqrt(r1)*(1-r2)*p2+np.sqrt(r1)*r2*p3
    return area, prob, uniform_pc

In [3]:
def farthest_point_sampling(pc, sample_num):
    # -------- TODO -----------
    # FOR LOOP is allowed here.
    # -------- TODO -----------
    n=pc.shape[0]
    res=[]
    init_idx=random.randint(0,n)
    res.append(init_idx)
    dis=np.full((n,),np.inf)
    pre_idx=init_idx

    cnt=1
    while cnt<sample_num:
        max_idx=-1
        max_dis=-1
        for i in range(n):
            t_dis=np.sqrt(np.sum((pc[i]-pc[res[-1]])**2))
            if t_dis<dis[i]:
                dis[i]=t_dis
        max_idx=np.argmax(dis)
        res.append(max_idx)
        pre_idx=max_idx
        cnt+=1
        
    results=pc[res]
    return results

In [4]:
# task 1: uniform sampling 

obj_path = 'spot.obj'
mesh = trimesh.load(obj_path)
print('faces shape: ', mesh.faces.shape)
sample_num = 512
area, prob, uniform_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, sample_num)

# Visualization. For you to check your code
np.savetxt('uniform_sampling_vis.txt', uniform_pc)

print('area shape: ',area.shape)
print('prob shape: ',prob.shape)
print('pc shape: ',uniform_pc.shape)
# the result should satisfy: 
#       area.shape = (13712, ) 
#       prob.shape = (13712, ) 
#       uniform_pc.shape = (512, 3) 

# For submission
save_dict = {'area': area, 'prob': prob, 'pc': uniform_pc}
np.save('../results/uniform_sampling_results', save_dict)

faces shape:  (13712, 3)
area shape:  (13712,)
prob shape:  (13712,)
pc shape:  (512, 3)


In [5]:
# task 2: FPS

init_sample_num = 2000
final_sample_num = 512
_,_, tmp_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, init_sample_num)
fps_pc = farthest_point_sampling(tmp_pc, final_sample_num)

# Visualization. For you to check your code
np.savetxt('fps_vis.txt', fps_pc)

# For submission
np.save('../results/fps_results', fps_pc)

In [6]:
# task 3: metrics

from earthmover.earthmover import earthmover_distance   # EMD may be very slow (1~2mins)
# -----------TODO---------------
# compute chamfer distance and EMD for two point clouds sampled by uniform sampling and FPS.
# sample and compute CD and EMD again. repeat for five times.
# save the mean and var.
# -----------TODO---------------


CD_mean = 0
CD_var = 0
EMD_mean = 0
EMD_var = 0


def chamfer_distance(pc1,pc2):
    cd=0
    n1,n2=pc1.shape[0],pc2.shape[0]
    arr1=np.tile(pc1,(n2,1))
    arr2=np.repeat(pc2,n1,axis=0)
    dis=np.sqrt(np.sum((arr1-arr2)**2,axis=1)).reshape(n2,n1)
    dis1=np.mean(np.min(dis,axis=0))
    dis2=np.mean(np.min(dis,axis=1))
    return (dis1+dis2)/2 # 除以二以和EMD比较

CD=[]
CD_test=[]
EMD=[]
init_sample_num=2000
final_sample_num=512
for i in range(5):
    _,_,pc1 = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, final_sample_num)
    _,_,tmp_pc = uniform_sampling_from_mesh(mesh.vertices, mesh.faces, init_sample_num)
    pc2 = farthest_point_sampling(tmp_pc, final_sample_num)
    CD.append(chamfer_distance(pc1,pc2))
    EMD.append(earthmover_distance(pc1,pc2))

CD_mean=np.mean(CD)
CD_var=np.var(CD)
EMD_mean=np.mean(EMD)
EMD_var=np.var(EMD)

print(CD)
print(EMD)
print(CD_mean,CD_var)
print(EMD_mean,EMD_var)

# For submission
np.save('../results/metrics', {'CD_mean':CD_mean, 'CD_var':CD_var, 'EMD_mean':EMD_mean, 'EMD_var':EMD_var})

[1.384855717419243, 1.3689680156506239, 1.3854134749137197, 1.3919746171143739, 1.374982591422476]
[2.6308866862762157, 2.1324462870232312, 2.7883147307123757, 2.0728163994016144, 2.4573023320115284]
1.3812388833040874 6.709601312916562e-05
2.416353287084993 0.07693549941551078
